In [0]:
!pip3 install pyshp
!pip3 install geopandas
!pip3 install OWSLib
!pip3 install zipfile

In [9]:
import shapefile as shp
import json
import geopandas as gpd
from owslib.wms import WebMapService

import zipfile

in the tutorial, the author gets the sattellite image data from the Dutch government open data portal. However, the data has been moved, and I got the rest of the data from the dropbox. I cannot get the satellite image data, so I just used his image tiles, but I know what he is doing. Now we will have to unzip our zipfiles containing the images and shapefiles:

In [11]:
#extracting the shapefiles
#zip_ref = zipfile.ZipFile('2017_09.zip')
#zip_ref.extractall('shapefile')

#extracting the image tiles
#zip_ref = zipfile.ZipFile('image_tiles_200.zip')
#zip_ref.extractall('image_tiles')

###here we define the BOUNDING_BOX, or region where we would like to visualize###

x_min = 90000 #min x-val (start in our coordinate)
y_min = 427000 #min y-val
dx, dy = 200, 200 #(the amount of x and y we would want stored from the map in each tile)

no_tiles_x = 100 #number of tiles on the grid over the image
no_tiles_y = 100 #total tiles we make is 10,000
 
x_max = x_min + no_tiles_x * dx #max x-val (end in our coordinates)
y_max = y_min + no_tiles_y * dy #max y-val 
BOUNDING_BOX = [x_min, y_min, x_max, y_max] #this is the important global variable we wanted



We extracted the zipfiles now. let's read our shapefile and convert its contents into a json so we do not have to repeat the shapefile-reading process each time:

In [12]:
filename ='shapefile/2017_09/Wegvakken.shp'

reader = shp.Reader(filename)

fields = reader.fields[1:] #list that stores each filed (or column)
field_names = [field[0] for field in fields]

buffer=[]
for sr in reader.shapeRecords(): #we should maybe use geopandas to read and save memory?
    attrib = dict(zip(field_names, sr.record)) #each column of the shp table gets in the 
    geom = sr.shape.__geo_interface__ #storing the info about the geometry in each row of the shaperecords
    buffer.append(dict(type="Feature", geometry=geom, properties=attrib))


We have now created the buffer list. The problem, now is that we need to make sure that some objects, such as bytes from the geometry part of the shapefile or datetime objects from the date column, need to be re-encoded, geopandas typically has a version of dealing with this by writing into a geojson.

In [0]:
def JSONencoder(obj):
    """JSON serializer for objects not serializable by default json code"""
    
    if isinstance(obj, (datetime, date)):
        serial = obj.isoformat()
        return serial
    if isinstance(obj, bytes):
        return {'__class__': 'bytes',
                '__value__': list(obj)}
    raise TypeError ("Type %s not serializable" % type(obj))

from this, we create create our json file

In [0]:
output_filename = 'output.json'
json_file = open(output_filename, "w")
json_file.write(json.dumps({"type": "FeatureCollection", "features": buffer}, indent=2, default=JSONencoder) + "\n") #write to json_file 
json_file.close()

Mapping the layers now. We want to determine which image tiles have roads, and which do not:

In [0]:
####### MAPPING TIME ##########
#let's define some important keys first
Python
#First we define some variables, and dictionary keys which are going to be used throughout the rest.
dict_roadtype = {
    "G": 'Gemeente',
    "R": 'Rijk',
    "P": 'Provincie',
    "W": 'Waterschap',
    'T': 'Andere wegbeheerder',
    '' : 'leeg'
}

dict_roadtype_to_color = {
    "G": 'red',
    "R": 'blue',
    "P": 'green',
    "W": 'magenta',
    'T': 'yellow',
    '' : 'leeg'
}

FEATURES_KEY = 'features'
PROPERTIES_KEY = 'properties'
GEOMETRY_KEY = 'geometry'
COORDINATES_KEY = 'coordinates'
WEGSOORT_KEY = 'WEGBEHSRT'

MINIMUM_NO_POINTS_PER_TILE = 4
POINTS_PER_METER = 0.1

INPUT_FOLDER_TILES = './data/image_tiles/'


filename_wegvakken = './data/nwb_wegvakken/2017_09_wegvakken.json'
dict_nwb_wegvakken = json.load(open(filename_wegvakken))[FEATURES_KEY]
d_tile_contents = defaultdict(list)
d_roadtype_tiles = defaultdict(set)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
#First we define some variables, and dictionary keys which are going to be used throughout the rest.
dict_roadtype = {
    "G": 'Gemeente',
    "R": 'Rijk',
    "P": 'Provincie',
    "W": 'Waterschap',
    'T': 'Andere wegbeheerder',
    '' : 'leeg'
}
 
dict_roadtype_to_color = {
    "G": 'red',
    "R": 'blue',
    "P": 'green',
    "W": 'magenta',
    'T': 'yellow',
    '' : 'leeg'
}
 
FEATURES_KEY = 'features'
PROPERTIES_KEY = 'properties'
GEOMETRY_KEY = 'geometry'
COORDINATES_KEY = 'coordinates'
WEGSOORT_KEY = 'WEGBEHSRT'
 
MINIMUM_NO_POINTS_PER_TILE = 4
POINTS_PER_METER = 0.1
 
INPUT_FOLDER_TILES = '/image_tiles/'
 
 
filename_wegvakken = './data/nwb_wegvakken/2017_09_wegvakken.json'
dict_nwb_wegvakken = json.load(open(filename_wegvakken))[FEATURES_KEY]
d_tile_contents = defaultdict(list)
d_roadtype_tiles = defaultdict(set)